In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from delta import *
import logging

In [2]:
GOLDEN_TABLE_PATH = '/opt/data_lake/golden/house_prices'
PREDICTIONS_TABLE_PATH = '/opt/data_lake/golden/house_prices_predictions'
DRUID_GOLDEN_TABLE_PATH = '/opt/data_lake/golden/druid/house_prices'
DRUID_PREDICTIONS_TABLE_PATH = '/opt/data_lake/golden/druid/house_prices_predictions'

In [3]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [4]:
# Create Spark Session
builder = (
    SparkSession
    .builder
    .appName("real-estate-etl")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2598f2b0-98a2-454a-998b-21b92e459800;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
:: resolution report :: resolve 406ms :: artifacts dl 17ms
	:: modules in use:
	com.ibm.icu#icu4j;58.2 from central in [default]
	io.delta#delta-core_2.12;1.0.0 from central in [default]
	org.abego.treelayout#org.abego.treelayout.core;1.0.3 from central in [default]
	org.antlr#ST4;4.0.8 from central in [default]
	org.antlr#antlr-run

In [11]:
logger.info('Exporting golden house prices for Druid ...')
golden_data = (
    spark
    .read
    .format('delta')
    .load(GOLDEN_TABLE_PATH)
)

golden_data.coalesce(10).write.mode("overwrite").csv(DRUID_GOLDEN_TABLE_PATH, sep="\t")

2022-07-10 01:08:22,975 [INFO] Exporting golden house prices for Druid ...


In [5]:
logger.info('Exporting model predictions for Druid ...')
predictions_data = (
    spark
    .read
    .format('delta')
    .load(PREDICTIONS_TABLE_PATH)
)

predictions_data.coalesce(10).write.mode("overwrite").csv(DRUID_PREDICTIONS_TABLE_PATH, sep="\t")

2022-07-10 01:30:42,665 [INFO] Exporting model predictions for Druid ...
